In [1]:
import json
import pandas as pd

In [2]:
#######################################################
# Read csv file
#######################################################

df = pd.read_csv('./data/GB_processed.csv')
# df = pd.read_csv('./data/<Region>_processed.csv')
#######################################################
# Data Cleansing and Transfrom
#######################################################

#convert data type back to datetime
df["trending_date"] = pd.to_datetime(df["trending_date"],format="%Y-%m-%d")
df["publishedAt"] = pd.to_datetime(df["trending_date"],format="%Y-%m-%d")
#rename column name to match style
df =df.rename(columns = {"publishedAt":"published_date"})

#add new column trending count that count how many times same video exist in trending
df["trending_count"]= df.groupby("video_id")["video_id"].transform("count")

#import category table from json file
category_id = {}

with open("./data/GB_category_id.json","r") as f:
    category = json.load(f)
for cat in category["items"]:
    category_id[int(cat["id"])] = cat["snippet"]["title"]
    
category = pd.DataFrame.from_dict(category_id,orient="index",columns = ["Category_name"])

#merge category df with base df
df = pd.merge(df,category,left_on ="categoryId",right_index =True)

#drop duplicate column, ***keep in mind we keep the last record as last record is the most updated record
df = df.drop_duplicates(subset =["video_id"],keep="last",inplace = False)


In [3]:
##############################################################################################################
# The most trending categroy , base on number of video belong to that category in trending list
##############################################################################################################

df.groupby(["categoryId","Category_name"])["video_id"].count().sort_values(ascending=False).reset_index().head(5)

,categoryId,Category_name,video_id
0,24,Entertainment,1456
1,20,Gaming,1359
2,17,Sports,1291
3,22,People & Blogs,751
4,10,Music,705


In [4]:
##############################################################################################################
# The most liked categroy , base on number of likes for each category in trending video
##############################################################################################################

df.groupby(["categoryId","Category_name"])["likes"].sum().sort_values(ascending=False).reset_index().head(5)

,categoryId,Category_name,likes
0,24,Entertainment,243385710
1,10,Music,229559828
2,20,Gaming,138457962
3,22,People & Blogs,98905908
4,23,Comedy,79917038


In [5]:
##############################################################################################################
# Top 5 tredning video base on views
##############################################################################################################

top5=df.sort_values(by = ["view_count"],ascending = False).head(5)

top5[["title","published_date","channelTitle","view_count","likes","country"]]

,title,published_date,channelTitle,view_count,likes,country
6487,LISA - 'LALISA' M/V,2021-09-16,BLACKPINK,154134590,11348979,GB
21692,"$456,000 Squid Game In Real Life!",2021-12-02,MrBeast,137068663,10926913,GB
693,MYSTERY INSIDE TOOTHPASTE?!,2021-08-18,Itzshauni,121633557,4650239,GB
13895,Adele - Easy On Me (Official Video),2021-10-23,AdeleVEVO,104017073,4149484,GB
27299,Crazy #alluarjun #painting #shorts #viral #tr...,2021-12-31,Dr.Harrsha Artist,87563278,5738628,GB


In [6]:
##############################################################################################################
# Comment to view ratio (Top5)
##############################################################################################################

df_comment_enable = df.drop(df[df["comments_disabled"] == True].index)

df_comment_enable["comment_to_view"] = (df_comment_enable["comment_count"]/df_comment_enable["view_count"])*100

df_comment_enable = df_comment_enable.sort_values(by ="comment_to_view",ascending = False ).head(5)

df_comment_enable[["title","published_date","channelTitle","view_count","comment_count","comment_to_view","Category_name","country"]]

,title,published_date,channelTitle,view_count,comment_count,comment_to_view,Category_name,country
4464,"Tory Lanez - Tuh (feat. EST Gee, VV$ Ken) [Off...",2021-09-06,Tory Lanez,1263117,161341,12.773243,Music,GB
32859,GIMME Your Questions!! I'm gonna animate the a...,2022-01-28,Let Me Explain Studios,588160,56297,9.571715,Film & Animation,GB
7895,"20,000,000 SUBSCRIBERS. (THE BIG GIVEAWAY)",2021-09-23,FaZe Rug,3023165,177283,5.864152,Entertainment,GB
19262,"Symonne Harrison, Nick Bencivengo - With You (...",2021-11-19,Symonne Harrison,549853,30511,5.548938,Music,GB
75,IS THIS THE END OF THE GOLF MATES ? 4K,2021-08-15,Golf Mates,82044,3921,4.779143,Sports,GB


In [7]:
##############################################################################################################
# The most commented categroy , base on number of likes for each category in trending video
##############################################################################################################

df.groupby(["categoryId","Category_name"])["comment_count"].sum().sort_values(ascending=False).reset_index().head(5)

,categoryId,Category_name,comment_count
0,10,Music,18329937
1,24,Entertainment,8659868
2,20,Gaming,8020457
3,17,Sports,2900197
4,22,People & Blogs,2894320
